In [ ]:
from bastionlab import Identity

# Create `Identity` for Data owner.
data_owner = Identity.create("data_owner")

In [ ]:
!wget 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'

In [ ]:
!python datagen.py

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
!unzip smsspamcollection.zip

In [ ]:
!pip install transformers==0.13.2

In [2]:
from tokenizers import Tokenizer
from bastionlab.polars.policy import Policy, Aggregation, Log
import polars as pl
from bastionlab import Connection

file_path = "./SMSSpamCollection"

labels = []
texts = []
with open(file_path) as f:
    for line in f.readlines():
        split = line.split("\t")
        labels.append(1 if split[0] == "spam" else 0)
        texts.append(split[1])
df = pl.DataFrame({"label": labels, "text": texts})

connection = Connection("localhost")


policy = Policy(safe_zone=Aggregation(min_agg_size=10), unsafe_handling=Log())
rdf = connection.client.polars.send_df(
    df.limit(int(df.height * 0.8)).limit(64), policy=policy, sanitized_columns=["text"]
)
test_rdf = connection.client.polars.send_df(
    df.tail(int(df.height * 0.2)).limit(64), policy=policy, sanitized_columns=["text"]
)


tokenizer = Tokenizer.from_pretrained("distilbert-base-uncased")
tokenizer.enable_truncation(max_length=32)
tokenizer.enable_padding(length=32)

train_rds = (
    rdf.convert(["text"], tokenizer.to_str())
    .collect()
    .to_dataset(["text_ids", "text_mask"], "label")
)
test_rds = (
    test_rdf.convert(["text"], tokenizer.to_str())
    .collect()
    .to_dataset(["text_ids", "text_mask"], "label")
)


train_rds._set_test_dataset(test_rds.train_dataset_ref)
test_rds.set_train_dataset(train_dataset=train_rds.train_dataset_ref)

In [ ]:
! pip install transformers

In [3]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from bastionlab.torch.utils import MultipleOutputWrapper

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False,
    torchscript=True,
)
model = MultipleOutputWrapper(model, 0)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier

In [4]:
remote_datasets = connection.client.torch.list_remote_datasets()
print(remote_datasets)

remote_datasets[0].trace_input

[tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0])]

In [5]:
from bastionlab.torch.optimizer_config import Adam
remote_learner = connection.client.torch.RemoteLearner(
    model,
    remote_datasets[0],
    max_batch_size=2,
    loss="cross_entropy",
    optimizer=Adam(lr=5e-5),
    model_name="DistilBERT",
)

Sending DistilBERT: 100%|████████████████████| 268M/268M [00:03<00:00, 68.0MB/s] 


In [6]:
remote_learner.fit(nb_epochs=2, eps=None)

GRPCException: Internal server error: code=StatusCode.INTERNAL message=Torch error: Internal torch error: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/bastionlab/torch/utils/___torch_mangle_101.py", line 18, in forward
    inner2 = self.inner
    distilbert = inner2.distilbert
    _0 = (distilbert).forward(input_ids, mask, )
          ~~~~~~~~~~~~~~~~~~~ <--- HERE
    _1 = torch.slice(_0, 0, 0, 9223372036854775807)
    x = torch.select(_1, 1, 0)
  File "code/__torch__/transformers/models/distilbert/modeling_distilbert/___torch_mangle_96.py", line 13, in forward
    transformer = self.transformer
    embeddings = self.embeddings
    _0 = (transformer).forward((embeddings).forward(input_ids, ), mask, )
                                ~~~~~~~~~~~~~~~~~~~ <--- HERE
    return _0
  File "code/__torch__/transformers/models/distilbert/modeling_distilbert/___torch_mangle_9.py", line 22, 